In [ ]:
# High-ram runtime helps in training, allows for num_workers = 4 instead of 2.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Fetch lib from drive
!cp -r ./gdrive/MyDrive/vocal-remover/lib/ ./lib

In [ ]:
# Get the base dataset split into training and validation set
!unzip ./gdrive/MyDrive/DSD_Split.zip -d ./

In [ ]:
# More appropriate name since we are not only using DSD data
!mv DSD100/ dataset/

mv: cannot stat 'DSD100/': No such file or directory


In [ ]:
from lib import nets

In [ ]:
!pip install -r ./gdrive/MyDrive/vocal-remover/requirements.txt

In [ ]:
import hopsworks
import torch
import os
import joblib
import hsml


In [ ]:
# connect with Hopsworks
project = hopsworks.login()

# get Hopsworks Model Registry
mr = project.get_model_registry()

# get model object, set wanted version or change to get_best_model(),
model = mr.get_model("vocal_remover", version=3)



In [ ]:
model_path = model.download()


In [ ]:
print(model_path)
!ls $model_path

In [ ]:
model_path_pth = model_path + "/vocal_model.pth"

In [ ]:
# Feature pipeline, check if the name of the module is still correct.
import youtube_downloader as ydwn

In [ ]:
# Youtube downloader command line tool
!pip install yt-dlp

In [ ]:
yd = ydwn.YoutubeDownloader()
yd.download()
yd.trim_files()

In [ ]:
# check amount of songs, should be same amount. 
# Important! Remember to fix so that every mixture has a match in instruments and vice versa, the downloader might not have gotten all mixtures/instrumentals.
!ls ./dataset/instruments | wc -l
!ls ./dataset/mixtures | wc -l

In [ ]:
# Temp ugly fix
!mv ./dataset/instruments/* ./dataset/training/instruments
!mv ./dataset/mixtures/* ./dataset/training/mixtures

In [ ]:
# required for the model to save
!mkdir models

In [ ]:
# train
!python3 ./gdrive/MyDrive/vocal-remover/train.py --dataset ./dataset/ --reduction_rate 0.5 --mixup_rate 0.5 --gpu 0 --epoch 30 --num_workers 4 --pretrained_model $model_path_pth --split_mode subdirs

In [ ]:
import os
model_dir = "vocal_remover_model"
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

In [ ]:
n_fft = 2048
modelType = nets.CascadedNet(n_fft, 32, 128)
# Choose the best performing model from models directory, check logs if uncertain which is best
# Example: 
best_model_path = "./models/model_iter24.pth"
new_model = modelType.load_state_dict(torch.load(best_model_path, map_location=torch.device('cpu')))


In [ ]:
# Save model to model_dir, the contents of model_di
torch.save(new_model.state_dict(), model_dir + "/vocal_model.pth" )

In [ ]:
print(model_path)
!ls $model_path

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import numpy as np

In [ ]:
# Use the data to create schema, input and output have same format
input_schema = Schema(np.load("./dataset/training/instruments/sr44100_hl1024_nf2048/001_inst.npy"))
output_schema = Schema(np.load("./dataset/training/instruments/sr44100_hl1024_nf2048/001_inst.npy"))
model_schema = ModelSchema(input_schema, output_schema)

In [ ]:
print(input_schema)
print(output_schema)
print(model_schema)

In [ ]:
# Create the model using the hopsworks model registry
# Remember to set correct version and validation loss, validation loss is available in training logs
vocal_model = mr.python.create_model(
        name="vocal_remover",
        # version=int(model_path[-1]) + 1,
        version = 3,
        metrics={"validation_loss": 0.000268},
        model_schema=model_schema,
        description="Remove vocal from song audio file",
    )

In [ ]:
vocal_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/4243/models/vocal_remover/3


Model(name: 'vocal_remover', version: 3)